In [25]:
# considering the instances only for which the rating is either one star or five star as dataset
# and classifying the rating on the basis of train data(either 1 or 5)

import pandas as pd
import numpy as np

In [26]:
data = pd.read_csv('data/yelp.csv')
data.shape

(10000, 10)

In [27]:
data.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [28]:
X = data['text']
y = data['stars']
print(X.shape)
print(y.shape)

(10000,)
(10000,)


In [45]:
X = data[(data.stars == 5) | (data.stars == 1)]
X = X['text']

In [46]:
y = data[(data.stars == 5) | (data.stars == 1)]
y = y['stars']

In [47]:
X.head()

0    My wife took me here on my birthday for breakf...
1    I have no idea why some people give bad review...
3    Rosie, Dakota, and I LOVE Chaparral Dog Park!!...
4    General Manager Scott Petello is a good egg!!!...
6    Drop what you're doing and drive here. After I...
Name: text, dtype: object

In [48]:
y.head()

0    5
1    5
3    5
4    5
6    5
Name: stars, dtype: int64

In [49]:
print(X.shape)

(4086,)


In [50]:
print(y.shape)

(4086,)


In [51]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [52]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

X_train_dtm = vect.fit_transform(X_train)

In [53]:
X_train_dtm.shape

(3064, 16825)

In [56]:
len(vect.get_feature_names())

16825

In [57]:
X_test_dtm = vect.transform(X_test)

In [58]:
X_test_dtm.shape

(1022, 16825)

In [59]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB().fit(X_train_dtm, y_train)



In [60]:
y_pred = nb.predict(X_test_dtm)

In [62]:
from sklearn import metrics

metrics.accuracy_score(y_test, y_pred)

0.9187866927592955

In [64]:
#y_pred_proba = nb.predict_proba(X_test_dtm)
metrics.confusion_matrix(y_test, y_pred)

array([[126,  58],
       [ 25, 813]])

In [65]:
# calculating the Null Accuracy
# Null accuracy : accuracy that can be achieved by predicting the most frequent class

y_test.value_counts()


5    838
1    184
Name: stars, dtype: int64

In [68]:
y_test.value_counts('5')

5    0.819961
1    0.180039
Name: stars, dtype: float64

In [70]:
null_accuracy = 838/len(y_test)
null_accuracy

0.8199608610567515

In [78]:
print(len(X_test[(y_test == 1) & (y_pred == 5)])) # False Positive
X_test[(y_test == 1) & (y_pred == 5)]

58


2175    This has to be the worst restaurant in terms o...
1781    If you like the stuck up Scottsdale vibe this ...
2674    I'm sorry to be what seems to be the lone one ...
9984    Went last night to Whore Foods to get basics t...
3392    I found Lisa G's while driving through phoenix...
8283    Don't know where I should start. Grand opening...
2765    Went last week, and ordered a dozen variety. I...
2839    Never Again,\nI brought my Mountain Bike in (w...
321     My wife and I live around the corner, hadn't e...
1919                                         D-scust-ing.
2490    Lazy Q CLOSED in 2010.  New Owners cleaned up ...
9125    La Grande Orange Grocery has a problem. It can...
9185    For frozen yogurt quality, I give this place a...
436     this another place that i would give no stars ...
2051    Sadly with new owners comes changes on menu.  ...
1721    This is the closest to a New York hipster styl...
3447    If you want a school that cares more about you...
842     Boy is

In [80]:
nb.feature_count_

array([[26.,  4.,  1., ...,  0.,  0.,  0.],
       [39.,  5.,  0., ...,  1.,  1.,  1.]])

In [82]:
one_token_count = nb.feature_count_[0,:]
one_token_count

array([26.,  4.,  1., ...,  0.,  0.,  0.])

In [84]:
five_token_count = nb.feature_count_[1,:]
five_token_count

array([39.,  5.,  0., ...,  1.,  1.,  1.])

In [91]:
tokens = pd.DataFrame({'token':vect.get_feature_names(), 'one_star':one_token_count, 'five_star':five_token_count}).set_index('token')
tokens.head()

,five_star,one_star
token,,
00,39.0,26.0
000,5.0,4.0
00a,0.0,1.0
00am,2.0,3.0
00pm,4.0,1.0


In [92]:
tokens.sample(5, random_state=6)

,five_star,one_star
token,,
fourteen,0.0,1.0
bean,38.0,7.0
student,6.0,0.0
cerignola,1.0,0.0
bethany,2.0,0.0


In [93]:
nb.class_count_

array([ 565., 2499.])

In [95]:
nb.class_count_[1]

2499.0

In [101]:
tokens['five_star'] = tokens.five_star + 1
tokens['one_star'] = tokens.one_star + 1

tokens['five_star'] = tokens.five_star / nb.class_count_[0]
tokens['one_star'] = tokens.one_star / nb.class_count_[1]

tokens.sample(5, random_state=6)

,five_star,one_star
token,,
fourteen,0.001773,0.0004
bean,0.001773,0.0004
student,0.001773,0.0004
cerignola,0.001773,0.0004
bethany,0.001773,0.0004


In [102]:
tokens['five_to_one_ratio'] = tokens.five_star / tokens.one_star

In [103]:
tokens.sample(5, random_state=6)

,five_star,one_star,five_to_one_ratio
token,,,
fourteen,0.001773,0.0004,4.429077
bean,0.001773,0.0004,4.429599
student,0.001773,0.0004,4.429161
cerignola,0.001773,0.0004,4.429092
bethany,0.001773,0.0004,4.429106


In [104]:
tokens.sort_values('five_to_one_ratio', ascending=False)

,five_star,one_star,five_to_one_ratio
token,,,
the,0.001851,0.000401,4.620000
and,0.001831,0.000400,4.571573
to,0.001808,0.000400,4.515049
of,0.001798,0.000400,4.491262
is,0.001797,0.000400,4.489326
it,0.001797,0.000400,4.487111
was,0.001793,0.000400,4.476960
in,0.001792,0.000400,4.476685
for,0.001791,0.000400,4.474296


In [107]:
# 10 tokens more often present in 5 stars
tokens.head(10)

,five_star,one_star,five_to_one_ratio
token,,,
00,0.001773,0.0004,4.429600
000,0.001773,0.0004,4.429144
00a,0.001773,0.0004,4.429077
00am,0.001773,0.0004,4.429104
00pm,0.001773,0.0004,4.429133
01,0.001773,0.0004,4.429105
02,0.001773,0.0004,4.429077
03,0.001773,0.0004,4.429077
03342,0.001773,0.0004,4.429077


In [108]:
# 10 tokens more often present in single rating 
tokens.tail(10)

,five_star,one_star,five_to_one_ratio
token,,,
zucchini,0.001773,0.0004,4.429216
zuchinni,0.001773,0.0004,4.429091
zumba,0.001773,0.0004,4.429120
zupa,0.001773,0.0004,4.429092
zuzu,0.001773,0.0004,4.429120
zwiebel,0.001773,0.0004,4.429092
zzed,0.001773,0.0004,4.429092
éclairs,0.001773,0.0004,4.429092
école,0.001773,0.0004,4.429092
